In [ ]:
import pandas as pd
import numpy as np

In [ ]:
complete_event_log = pd.read_csv('./data/eventlog.csv').dropna(subset=['actie_omschrijving'])
complete_event_log

In [ ]:
event_log = pd.read_csv('./data/BPM_event_log_translated.csv', index_col=0)

In [ ]:
labels = pd.read_csv('./data/labels.csv', index_col = 0)
labels

In [ ]:
event_log[event_log.dossier_nr == 11848531]

In [ ]:
event_log = event_log.merge(labels, on='dossier_nr', how='left')
event_log = event_log[event_log['duration'] >= 0]
event_log

In [ ]:
for col in ['datum_van', 'datum_tot', 'datum_actie']:
    event_log[col] = pd.to_datetime(event_log[col])
    complete_event_log[col] = pd.to_datetime(complete_event_log[col])
    

In [ ]:
event_log['days_in_execution_phase'] = (event_log['datum_actie'] - event_log['datum_van']).dt.days
event_log['days_in_execution_phase']

In [ ]:
pipeie = pd.get_dummies(event_log.action, prefix='action')
pipeie

In [ ]:
event_log.columns

In [ ]:
merged = event_log[['dossier_nr', 'duration', 'days_in_execution_phase']].merge(pipeie, how='inner', left_index=True, right_index=True)
merged

In [ ]:
merged[merged.duration < 0]

In [ ]:
bieboep = merged[merged.days_in_execution_phase < 30]

In [ ]:
bieboep.groupby('dossier_nr').sum()

In [ ]:
merged

In [ ]:
event_log.groupby('dossier_nr').count().describe()

In [ ]:
event_log.dossier_nr.nunique()

In [ ]:
event_log[event_log.days_in_execution_phase < 120].dossier_nr.nunique()

## Landmarking

In [ ]:
event_log[event_log.duration >= 0].dossier_nr.nunique()

In [ ]:
dataset = pd.read_csv('./data/dataset.csv', index_col = 0)
dataset

In [ ]:
labels = pd.read_csv('./data/labels.csv', index_col = 0)
labels = labels[labels.duration >= 0]
labels

In [ ]:
complete_event_log

In [ ]:
labels

In [ ]:
complete_event_log = complete_event_log.merge(dataset.reset_index()[['dossier_nr', 'debiteur_relatie_nr']], how='inner', left_on='dossier_nr', right_on='dossier_nr')

In [ ]:
complete_dataset = pd.DataFrame()

for i in range(0, labels.duration.max(), 90):
# for i in [0, 30]:
    dossiers = labels[labels.duration >= i]
    altered_labels = labels.copy()
    altered_labels['duration'] = altered_labels['duration'] - i
    
    filtered = merged[(merged.dossier_nr.isin(dossiers.index)) & (merged.days_in_execution_phase <= i)]
    
    grouped = filtered.groupby('dossier_nr').sum()
    filtered_dataset = dataset[dataset.index.isin(dossiers.index)]
    
    
    res = filtered_dataset.merge(grouped, on='dossier_nr', how='left').fillna(0).reset_index()
    res['days_in_execution_phase'] = i
    
    res = res.drop(['duration', 'action_Payment'], axis = 1)
    res = res.merge(altered_labels, on='dossier_nr', how='inner')


    
    
    complete_dataset = complete_dataset.append(res)

In [ ]:
complete_dataset = complete_dataset.reset_index()

In [ ]:
complete_dataset['days_in_execution_phase'].value_counts()

In [ ]:
complete_dataset.columns

In [ ]:
complete_dataset

In [ ]:
from datetime import timedelta

In [ ]:
complete_event_log

In [ ]:
complete_event_log.datum_actie = complete_event_log.datum_actie.dt.date

In [ ]:
complete_dataset

In [ ]:
fc_columns = ['nr_cases_in_execution_phase' , 'nr_open_cases_in_execution_phase', 'nr_cases_with_payment', 'nr_cases_without_payment', 'debtor_payment_past_30_days', 'debtor_payment_past_180_days', 'debtor_payment_past_365_days', 'debtor_payment_in_dataset', 'debtor_contact_past_30_days', 'debtor_contact_past_180_days', 'debtor_contact_past_365_days', 'debtor_contact_in_dataset', 'debtor_official_acts', 'debtor_outgoing_emails', 'debtor_outgoing_letters', 'debtor_outbound_calls', 'debtor_incoming_emails', 'debtor_incoming_letters', 'debtor_incoming_calls']
feature_creation_df = pd.DataFrame(columns = fc_columns)
for i, dossier in complete_dataset.iterrows():
#     print(dossier)
    debtor_nr = dossier.debiteur_relatie_nr
    complete_case_event_log = complete_event_log[complete_event_log.dossier_nr == dossier.dossier_nr]
    current_date = complete_case_event_log.datum_van.min() + timedelta(dossier.days_in_execution_phase)
#     print(current_date - timedelta(30))
#     print(current_date)
#     current_date += timedelta(10000)
    
    
    debtor_cases = complete_event_log[(complete_event_log.debiteur_relatie_nr == debtor_nr) & (complete_event_log.datum_actie <= current_date)]
    nr_cases = debtor_cases.dossier_nr.nunique()
    open_debtor_cases = debtor_cases[debtor_cases['datum_tot'] > current_date]
    nr_open_cases = open_debtor_cases.dossier_nr.nunique()
    
    payments = debtor_cases[(debtor_cases['bedrag_betaling'] > 0) & (debtor_cases['cat_actie_code'] != 999.0)]
    
    nr_cases_with_payment = payments.dossier_nr.nunique()
    nr_cases_without_payment = nr_cases - nr_cases_with_payment
    
    payment_30_days = payment_180_days = payment_one_year = payment_in_dataset = 0
    
    if len(payments):
        days_since_last_payment = (current_date.date() - payments.datum_actie.max()).days
#         print('days_since_last_payment {}'.format(days_since_last_payment))
        if days_since_last_payment <= 30:
            payment_30_days = 1
        if days_since_last_payment <= 180:
            payment_180_days = 1
        if days_since_last_payment <= 365:
            payment_one_year = 1
        payment_in_dataset = 1
#     if len(payments)

    debtor_contacts = debtor_cases[((debtor_cases.cat_actie_code >= 200.0) & (debtor_cases.cat_actie_code <= 230.0)) | ((debtor_cases.cat_actie_code == 270.0) & (debtor_cases.actie_code == 225.0))]
    
    contact_30_days = contact_180_days = contact_one_year = contact_in_dataset = 0
    if len(debtor_contacts):
        days_since_last_contact = (current_date.date() - debtor_contacts.datum_actie.max()).days
        if days_since_last_contact <= 30:
            contact_30_days = 1
        if days_since_last_contact <= 180:
            contact_180_days = 1
        if days_since_last_contact <= 365:
            contact_one_year = 1
        contact_in_dataset = 1
    
    
    nr_official_acts = len(debtor_cases[debtor_cases.cat_actie_code == 790.0])
    nr_outgoing_emails = len(debtor_cases[debtor_cases.cat_actie_code == 260.0])
    nr_outgoing_letters = len(debtor_cases[debtor_cases.cat_actie_code == 250.0])
    nr_outbound_calls = len(debtor_cases[(debtor_cases.cat_actie_code == 270.0) & (debtor_cases.actie_code == 225.0)])
    
    nr_incoming_emails = len(debtor_cases[debtor_cases.cat_actie_code == 230.0])
    nr_incoming_letters = len(debtor_cases[debtor_cases.cat_actie_code == 200.0])
    nr_incoming_calls = len(debtor_cases[(debtor_cases.cat_actie_code == 220.0)])
#     print(nr_cases, nr_open_cases)
#     print(len(payments))


    feature_creation_df = feature_creation_df.append(pd.DataFrame([[nr_cases, nr_open_cases, nr_cases_with_payment, nr_cases_without_payment, payment_30_days, payment_180_days, payment_one_year, payment_in_dataset, contact_30_days, contact_180_days, contact_one_year, contact_in_dataset, nr_official_acts, nr_outgoing_emails, nr_outgoing_letters, nr_outbound_calls, nr_incoming_emails, nr_incoming_letters, nr_incoming_calls]], columns = fc_columns))
    
    if i % 500 == 0:
        print(i)
#     if nr_cases == 81:
#         break
#     if len(payments) > 500:
#         break
#     if i > 100:
#         break


In [ ]:
feature_creation_df = feature_creation_df.astype(int)

In [ ]:
# complete_event_log[(complete_event_log.debiteur_relatie_nr == 887541) & (complete_event_log.cat_actie_code == 220.0)]

In [ ]:
feature_creation_df

In [ ]:
feature_creation_df.describe()

In [ ]:
feature_creation_df.to_csv('./data/feature_creation_landmarking.csv')

In [ ]:
feature_creation_df['index'].describe()

In [ ]:
feature_creation_df.columns.sort_values()

In [ ]:
feature_creation_df = pd.read_csv('./data/feature_creation_landmarking.csv', index_col=0)

In [ ]:
(feature_creation_df.nr_cases_in_execution_phase - feature_creation_df.nr_open_cases_in_execution_phase).describe()

In [ ]:
feature_creation_df.astype(int).describe()

In [ ]:
feature_creation_df.drop(['index'], axis=1, inplace=True)
complete_dataset.drop(['index'], axis=1, inplace=True)

In [ ]:
complete_dataset = complete_dataset.merge(feature_creation_df, left_index=True, right_index=True)

In [ ]:
complete_dataset

In [ ]:
nr_official_acts

In [ ]:
nr_outgoing_emails

In [ ]:
nr_outgoing_letters

In [ ]:
nr_outbound_calls

In [ ]:
payment_in_dataset

In [ ]:
nr_cases

In [ ]:
nr_open_cases

In [ ]:
payment_one_year

In [ ]:
payment_30_days

In [ ]:
current_date

In [ ]:
dossier

In [ ]:
complete_dataset.to_csv('./data/complete_dataset.csv')

In [ ]:
complete_dataset[complete_dataset.dossier_nr == 11800412]

In [ ]:
complete_dataset